In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/JKTech/'Llama with Custom FineTuning'

departments.csv  LLama.ipynb  products.csv


Insta Cart Data : https://www.kaggle.com/competitions/instacart-market-basket-analysis/data

In [4]:
import pandas as pd
df_product = pd.read_csv("/content/drive/MyDrive/JKTech/Llama with Custom FineTuning/products.csv")
df_dept = pd.read_csv("/content/drive/MyDrive/JKTech/Llama with Custom FineTuning/departments.csv")


In [5]:
df_product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
df_dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [7]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])
df_joined['text'] = df_joined.apply(lambda row: row['product_name'] + " ->: " + row['department'], axis = 1)

In [8]:
df_joined.head()

,product_id,product_name,aisle_id,department_id,department,text
0,1,Chocolate Sandwich Cookies,61,19,snacks,Chocolate Sandwich Cookies ->: snacks
1,16,Mint Chocolate Flavored Syrup,103,19,snacks,Mint Chocolate Flavored Syrup ->: snacks
2,25,Salted Caramel Lean Protein & Fiber Bar,3,19,snacks,Salted Caramel Lean Protein & Fiber Bar ->: sn...
3,32,Nacho Cheese White Bean Chips,107,19,snacks,Nacho Cheese White Bean Chips ->: snacks
4,41,Organic Sourdough Einkorn Crackers Rosemary,78,19,snacks,Organic Sourdough Einkorn Crackers Rosemary ->...


In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=0.2, random_state=42)

In [10]:
train_df.head()

,product_id,product_name,aisle_id,department_id,department,text
7361,10376,Organic Caraway Seeds,104,13,pantry,Organic Caraway Seeds ->: pantry
32534,40131,Original Laundry Detergent,75,17,household,Original Laundry Detergent ->: household
1457,11913,Shelled Pistachios,117,19,snacks,Shelled Pistachios ->: snacks
5201,41392,Harvest Berry Chewy Granola Bars,3,19,snacks,Harvest Berry Chewy Granola Bars ->: snacks
38539,28627,Veganic Sprouted Red Fife Raisin Bran,121,14,breakfast,Veganic Sprouted Red Fife Raisin Bran ->: brea...


In [11]:
test_df.head()

,product_id,product_name,aisle_id,department_id,department,text
33626,24154,Free & Clear Stage 4 Overnight Diapers,56,18,babies,Free & Clear Stage 4 Overnight Diapers ->: babies
18192,27348,"Beef pot roast with roasted potatoes, carrots,...",38,1,frozen,"Beef pot roast with roasted potatoes, carrots,..."
47099,27181,Coffee Liquer,124,5,alcohol,Coffee Liquer ->: alcohol
48183,20577,Bread Rolls,43,3,bakery,Bread Rolls ->: bakery
22197,16472,French Milled Oval Almond Gourmande Soap,25,11,personal care,French Milled Oval Almond Gourmande Soap ->: p...


In [12]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [14]:
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [17]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")
    print()

Result: “Free & Clear Stage 4 Overnight Diapers” ->:
 październik 1, 2014 w 9:55 pm	 Wpis można edytować
[…] the last few years, I’ve become pretty good at stockpiling diapers. I’ve written posts about the best prices on diapers, how to organize your stockpile, and how to save […]
[…] a few posts ago, I wrote about how I stockpile diapers, including a diaper stockpile organizer that helps us manage the stash. Today, I want to share how I use […]
[…] a few posts ago (read it here), I wrote how I stockpile diapers, including a diaper stockpile organizer that helps us manage the stash. We use a different […]
[…] of you have read my posts about how

Result: Bread Rolls ->:
 Einzeln, 110g
1.25 x Kartofelpuffer ->:
Kartoffelsuppe
Kartoffelmus
Fertiger Brei ->:
Fertiger Brei
Einzeln, 125 g
1.50 x Pommes frites ->:
1.50 x Bratwurst mit Gemüse
1.75 x Pommes frites ->:
1.75 x Schnitzel
1.50 x Bratkartoffeln ->:
1.75 x Bratfisch im Knoblauch
2.00 x Fischfilet
2.25 x Bratkartoffeln
1.75 x Fischfil

In [18]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

In [19]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [20]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [21]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [22]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.029100
2,3.089600
3,4.359200
4,6.271200
5,4.250100
6,3.092300
7,4.517100
8,3.968700
9,4.470100
10,5.355700


TrainOutput(global_step=120, training_loss=2.698847731947899, metrics={'train_runtime': 527.5984, 'train_samples_per_second': 3.639, 'train_steps_per_second': 0.227, 'total_flos': 627303342243840.0, 'train_loss': 2.698847731947899, 'epoch': 0.05})

In [23]:
lst_test_data = list(test_df['text'])

In [24]:
len(lst_test_data)

9938

In [25]:
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [26]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])
    print()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0 Free & Clear Stage 4 Overnight Diapers ->: babies: care: personal hygiene: diapers: cloth & disposable ->: baby care: diapers: diapers for kids: overnight diapers ->: personal care: diapers & incontinence: diapers babies ->: personal care: diapers & incontinence: diapers: free & clear ->: babies: care: personal hygiene: dia

1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen foods ->: meals ->: main courses beef, pork meat, fish, seafood ->: meats, poultry ->: meat: steaks ->: beef steaks ->: beef meat: steaks ->: beef steaks: steak dishes ->:

2 Coffee Liquer ->: alcohol spirits coffee liqueur ->: beverages non-alcoholic beverages coffee beverages ->: coffee coffee beverages coffee beverages ->: alcoholic ->: spirits beverages ->: beverages non-alcoholic beverages coffee beverages coffee beverages ->: alcoholic ->: spirits beverages ->: beverages non-alcoholic bever

3 Bread Rolls ->: bakery ->: snacks ->: breakfast

In [27]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies: care: personal hygiene: diapers: cloth & disposable',
 'frozen foods',
 'alcohol spirits coffee liqueur',
 'bakery',
 'personal care beauty soaps soap bar',
 'household: cleaning',
 'deli meat & cheese',
 'beverages: soft drinks & waters',
 'dairy eggs & dairy ice cream dairy products butter',
 'canned goods canned soups',
 'dairy eggs cheese eggs & dairy',
 'snacks candy bars',
 'dry goods pasta',
 'personal care',
 'beverages',
 'international pantry: dry goods: spices',
 'breakfast',
 'dairy eggs eggs',
 'meat seafood',
 'household cleaning household',
 'frozen foods',
 'dairy eggs & cheese eggs & dairy eggs & beans',
 'breakfast',
 'personal care',
 'meat seafood']

In [28]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies: care: personal hygiene: diapers: cloth...
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen foods
2,Coffee Liquer,alcohol,alcohol spirits coffee liqueur
3,Bread Rolls,bakery,bakery
4,French Milled Oval Almond Gourmande Soap,personal care,personal care beauty soaps soap bar
5,Dust Pan,household,household: cleaning
6,Roasted Pine Nut Hommus,deli,deli meat & cheese
7,Cranberry Raspberry Juice Cocktail,beverages,beverages: soft drinks & waters
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs & dairy ice cream dairy products bu...
9,Traditional Chicken Barley Soup,canned goods,canned goods canned soups
